In [1]:
import polars as pl

In [2]:
# Import incidents and format columns
df = pl.read_csv(
    "./data/incident_dump.csv",
).with_columns(
    pl.col("reported")
    .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%z")
    .dt.convert_time_zone("America/Chicago"),
    pl.col("reported_date").str.to_date("%Y-%m-%d"),
    pl.col("validated_location").str.split(",").cast(pl.List(pl.Float64)),
    pl.col("incident").str.to_lowercase(),
)

In [4]:
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""theft""",2968
"""information""",1378
"""found property…",918
"""lost property""",835
"""traffic violat…",534
"""information / …",499
"""liquor law vio…",443
"""medical call""",428
"""battery""",421


In [5]:
excluded_list = [
    "Fondling",
    "Medical Call",
    "Luring a Minor",
    "Lost Property",
    "Stalking",
    "Sexual Assault",
    "Dating",
    "Stalking",
    "Domestic",
    "Sex",
    "Found Property",
    "Mental Health",
    "Harassment by Electronic Means",
    "Well-Being",
    "Threatening Phone Call",
    "Medical Transport",
    "Warrant",
    "Lost Wallet",
    "Fire Alarm",
    "Chemical Spill",
    "Suspicious Mail",
    "Eavesdropping",
    "Sex Offense",
    "Sex Offender",
    "Sex Crime",
    "Domestic Aggravated Battery",
    "Dating Violence",
    "Harassing Messages",
]
df = df.filter(~pl.col("incident").str.contains("|".join(excluded_list)))
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""theft""",2968
"""information""",1378
"""found property…",918
"""lost property""",835
"""traffic violat…",534
"""information / …",499
"""liquor law vio…",443
"""medical call""",428
"""battery""",421


In [6]:
def list_to_parsed_set(unparsed_list: [str]):
    unparsed_list.sort()
    parsed_set = set()
    for element in unparsed_list:
        if "/" in element:
            for p in element.split(r"\s?/\s?"):
                fmt_element = p.strip().lower()
                if p:
                    parsed_set.add(fmt_element)
        else:
            fmt_element = element.strip().lower()
            parsed_set.add(fmt_element)
    return parsed_set

In [7]:
print(list_to_parsed_set(df["incident"].to_list()))

{'reckless discharge of firearm', 'disorderly conduct / telephone threat', 'robbery-strong arm', 'arson by fire', 'information report / disturbance', 'information / missing person found', 'information report / stuck elevator', 'aggravated battery of a police officer', 'weapon violation', 'assist other agency / shot fired', 'sex offender / prohibited zone', 'deceptive practice / fraud', 'violation of order of protection', 'non-criminal damage to vehicle', 'traffic crash / hit and run / property damage only', 'theft of service', 'information / arrest', 'non-criminal / death investigation', 'attempted theft', 'deceptive practice / credit card fraud', 'traffic crash / hit and run property damage', 'shot fired / criminal damage to vehicle', 'information / missing person', 'found drug paraphernalia', 'lost wallet', 'unlawful possession of a handgun', 'assist other agency / attempted armed robbery', 'sick person', 'deceptive practice / theft of lost or mislaid property', 'unlawful possession 